In [1]:
from tensorboard.backend.event_processing import event_accumulator
import json


def tb_get(path, tag, step_cutoff):
    # read tensorboard scalar data

    ea = event_accumulator.EventAccumulator(path, size_guidance={"scalar": 0})
    ea.Reload()
    scalar_tensor = ea.Scalars(tag)
    target = min(step_cutoff, len(scalar_tensor) - 1)

    return scalar_tensor[target].value


def check_metrics(tag, folder, key, step):
    asr_cnt = 0
    retrieved_cnt = 0
    retrieved_idx = []
    llm_loss = 0
    retriever_loss = 0
    optimize_idx = []

    try:
        asr_cnt = tb_get(f"{folder}", "test/called", step)
        retrieved_cnt = tb_get(f"{folder}", "test/retrieved", step)
    except Exception as e:
        pass

    try:
        llm_loss = tb_get(f"{folder}", "loss/decoder", step)
        retriever_loss = tb_get(f"{folder}", "loss/encoder", step)
        ea = event_accumulator.EventAccumulator(folder, size_guidance={"tensors": 0})
        ea.Reload()
        retrieved_idx = ea.Tensors("retrieved_idxs/text_summary")
        retrieved_idx = json.loads(
            retrieved_idx[step].tensor_proto.string_val[0].decode("utf-8")
        )
        optimize_idx = ea.Tensors("success_idxs/text_summary")
        optimize_idx = json.loads(
            optimize_idx[step].tensor_proto.string_val[0].decode("utf-8")
        )
    except Exception as e:
        pass
    return {
        "asr_cnt": asr_cnt,
        "retrieved_cnt": retrieved_cnt,
        "retrieved_idx": list(retrieved_idx),
        "llm_loss": llm_loss,
        "retriever_loss": retriever_loss,
        "optimize_idx": list(optimize_idx),
    }

In [ ]:
clusters = list(range(3))
objs = 0
steps = [4, 8, 16, 32]


def fetch_result(cluster=0, obj=0):
    base_dir = f"logs/prag_base_dq/tb/prag_base_dq.cluster_{cluster}_obj_{obj}"
    v2_dir = f"logs/prag_v2_dq/tb/prag_v2_dq.cluster_{cluster}_obj_{obj}"

    non_optimize_idxs = set(
        check_metrics("phantom_base_dq", base_dir, "test/called", 256 // 4)[
            "optimize_idx"
        ]
    )

    baseline_asr = []
    v2_asr = []
    baseline_retrieved = []
    v2_retrieved = []
    baseline_asr_optimize = []
    v2_asr_optimize = []
    for step in steps:
        result_baseline = check_metrics(
            "phantom_base_dq", base_dir, "test/called", (step + 256) // 4
        )
        result_v2 = check_metrics("phantom_v2_dq", v2_dir, "test/called", step // 4)
        print(f"step: {step}")
        print(
            f"baseline: {result_baseline['asr_cnt']}, {result_baseline['retrieved_cnt']}, {len(result_baseline['retrieved_idx'])}"
        )
        print(
            f"v2: {result_v2['asr_cnt']}, {result_v2['retrieved_cnt']}, {len(result_v2['retrieved_idx'])}"
        )
        all_idxs = set(range(25))
        optimize_idxs = all_idxs - non_optimize_idxs
        print(f"optimize: {optimize_idxs}")
        base_optimize_idxs = set(result_baseline["optimize_idx"]) & optimize_idxs
        v2_optimize_set = set(result_v2["optimize_idx"]) & optimize_idxs
        print(f"baseline: {len(base_optimize_idxs)}, {len(optimize_idxs)}")
        print(f"v2: {len(v2_optimize_set)}, {len(optimize_idxs)}")

        baseline_asr.append(result_baseline["asr_cnt"])
        v2_asr.append(result_v2["asr_cnt"])
        baseline_retrieved.append(result_baseline["retrieved_cnt"])
        v2_retrieved.append(result_v2["retrieved_cnt"])
        baseline_asr_optimize.append(len(base_optimize_idxs))
        v2_asr_optimize.append(len(v2_optimize_set))

    return {
        "baseline_asr": baseline_asr,
        "v2_asr": v2_asr,
        "baseline_retrieved": baseline_retrieved,
        "v2_retrieved": v2_retrieved,
        "baseline_asr_optimize": baseline_asr_optimize,
        "v2_asr_optimize": v2_asr_optimize,
        "optimize_count": len(optimize_idxs),
    }


result = {}

for cluster in clusters:
    for obj in objs:
        result[(cluster, obj)] = fetch_result(cluster, obj)

In [ ]:
from openpyxl import Workbook

wb = Workbook()
ws = wb.create_sheet(title=f"cluster_{cluster}")

for cluster in clusters:
    ws.append(["cluster", cluster])
    for obj in objs:
        result_obj = result[(cluster, obj)]

        # Append result tables
        ws.append(["obj", obj])
        ws.append(["step"] + steps)
        ws.append(["baseline_asr"] + result_obj["baseline_asr"])
        ws.append(["v2_asr"] + result_obj["v2_asr"])
        ws.append(["baseline_retrieved"] + result_obj["baseline_retrieved"])
        ws.append(["v2_retrieved"] + result_obj["v2_retrieved"])
        ws.append(["baseline_asr_optimize"] + result_obj["baseline_asr_optimize"])
        ws.append(["v2_asr_optimize"] + result_obj["v2_asr_optimize"])
        ws.append(["optimize_count", result_obj["optimize_count"]])

        # Calculate ASRs
        optimize_count = result_obj["optimize_count"]
        ws.append([])  # Add a blank row for separation
        ws.append(["ASR"])
        ws.append(["obj", obj])
        ws.append(["step", 0] + steps)
        ws.append(
            ["baseline_asr", 1 - result_obj["optimize_count"] / 25]
            + [x / 25 for x in result_obj["baseline_asr"]]
        )
        ws.append(
            ["v2_asr", 1 - result_obj["optimize_count"] / 25]
            + [x / 25 for x in result_obj["v2_asr"]]
        )
        ws.append(
            ["baseline_optimize_asr", 0]
            + [x / optimize_count for x in result_obj["baseline_asr_optimize"]]
        )
        ws.append(
            ["v2_optimize_asr", 0]
            + [x / optimize_count for x in result_obj["v2_asr_optimize"]]
        )

        ws.append([])
        ws.append([])


wb.save("asr_results.xlsx")